<a href="https://breakingbad.echosystem.fr/">![Breaking DDES](dados/titulo.png)</a>




*v3M sEr hAcKuDã0 4m19uiNh0*

Este `notebook` é sobre criptografia DES e DDES e formas de quebrá-las. Espera-se que o H4cKuD0 que está lendo complete os exercícios para que ele faça sentido. Se estiver com preguiça, uma versão já resolvida do `notebook` também está disponível. Confira em: https://github.com/luxedo/palestrinha/tree/master/breaking-ddes

# 0. Receita de Frango Frito

O Hank interceptou duas mensagens criptografadas do Gus. Uma delas parece uma receita, a outra não conseguimos identificar. O DEA ficou sabendo que o Gus não é um bom criptógrafo e está usando DES para criptografar as mensagens. Será que o conteúdo da segunda mensagem é capaz de incriminar Gus?

```python
cifra_gus1 = b'\x1c\xce\x12\xbagH7j\x03\xb1\xe7\x9a+p\xe0\x0b\xb8\r\xf5\\\xeb\xd2l\xab\xda\xa704\x9a\xa3\xba\nm4"$n\xfb\xfd\xaa\xe8\x88\x15ek.\xc8y\x05\x1c\x86w\xe1\xca"\xf0\xba\xe3\xe0\x1eF\x88/\xc9\xba\xcfB\x15\xfc\x9a\x0f)'
texto_gus1 = b"Uma galinha fatiada, creme de leite de ganso manso, sal, pimenta, ..."

cifra_gus2 = b'\xd6\xfdwc\xf4\xa2I\x16\x82\xb3`\x06\xcbO\x07i\x8d\x89\x88c\x17\xacV?;\xf5|\x0f\x1ab\x95\x14\x01k\x1fW(<\xca\x19T\x8b\x11\xe1\xa0x\xae(|\xf6\xbc\xf1\xc1\xb4\x8d\x11\xe2\xe4\x80\x8d\xd97\x1c\xa5UW\x97\xf7r\xcb\x98\xc51K\x1bv\xd6\x8av\xd8\x15\xe3X"\xa6\x9a*&'
texto_gus2 = ???
```
![gus hank](dados/gus-hank.png)

Pra quem não conhece, Gus Fring é um dos barões da Metanfetamina na série *Breaking Bad*, enquanto Hank Schrader é um agente do DEA, o departamento responsável pela repreensão e controle de narcóticos.

# 1. DES

* *Data Encryption Standard* (IBM - 1975)
* Cifra de blocos (Rede Feistel - 16 rodadas)
* Tamanho do bloco: 64 bits
* Tamanho da chave: 56 bits (72,057,594,037,927,936 chaves)
    * Chave quebrada em 1999 ([EFF DES Cracker](https://en.wikipedia.org/wiki/EFF_DES_cracker), Força bruta ~22h)
* Algoritmo lento

### Cifra de blocos
Uma cifra de blocos consiste em dois algoritmos, um para criptografar *E* e outro para descriptografar *D*. Ambos os algoritmos aceitam dois parâmetros: Um bloco de comprimento conhecido e uma chave. O resultado do algoritmo é um bloco com dados aparentemente randômicos.

A chave utilizada nas cifras de blocos é simétrica, ou seja, a mesma chave que faz a criptografia dos dados também faz a descriptografia.

O algoritmo *E* recebe como parâmetros um bloco de **texto** (*plaintext*) e a chave, retornando um bloco de **texto cifrado** (*ciphertext*). Já a função de descriptografar *D* faz a operação inversa.

![Cifra de Blocos](dados/DES-block.png)

> ### Cifra de blocos cheatsheet
> * Opera em blocos de tamanho conhecido
> * Usa chaves simétricas
> * [AES](https://en.wikipedia.org/wiki/Advanced_Encryption_Standard) é provavelmente a cifra de blocos mais utilizada no mundo.

### Rede de Feistel

[Computerphile](https://www.youtube.com/watch?v=FGhj3CGxl8I&ab_channel=Computerphile)

Para os curiosos, uma rede de Feistel, ou cifra de bloco Luby–Rackoff, é o algoritmo base do DES. O funcionamento está além do escopo desse `notebook`, então fica aí a ilustração.

<img src="dados/Feistel_cipher_diagram_en.svg" style="width:50%;"></img>

Um fato interessante da rede de Feistel é que a função de descriptografar *D* é a mesma de criptografar *E*, porém com as sub-chaves invertidas.

Também notem o uso do operador *XOR* ⊕ na cifra, esse operador é muito utilizado em criptografia.

### Exercício 1
Vamos usar a biblioteca PyCryptodome. O funcionamento das cifras está documentado [aqui](https://pycryptodome.readthedocs.io/en/latest/src/cipher/cipher.html#api-principles).
Para ambos os exercícios, use a chave b"1OCTANOL".

a) Criptografe o texto `b"ME VE UM ESCATOL"` na variável `cifrado`.

b) Descriptografe a mesma mensagem para recuperar o texto na variável `decifrado`.

In [ ]:
from Crypto.Cipher import DES

chave = b'1OCTANOL'
texto = b"ME VE UM ESCATOL"

des = DES.new(chave, DES.MODE_ECB)

print("Texto original: ", texto)

# IMPLEMENTE A PARTIR DAQUI!

# FIM DA IMPLEMENTAÇÃO

print("Texto cifrado:  ", cifrado)
assert cifrado == b'\xdf\xdb\xc7\xb27|\xbb\x9c<#0\xdfN\x8f;L', "Erro no exercício a"
print("Texto decifrado:", decifrado)
assert decifrado == texto, "Erro no exercício b"

Algumas moléculas, nada a ver com o assunto.

<table>
    <tr>
        <td>
          <img src="dados/octanol.png"></img>
        </td>
        <td>
          <img src="dados/metilamina.png"></img>
        </td>
        <td>
          <img src="dados/sulfurico.png"></img>
        </td>
     </tr>
     <tr>
        <td style="text-align: center;">
            <a href="https://en.wikipedia.org/wiki/1-Octanol">Octanol</a>
        </td>
        <td style="text-align: center;">
            <a href="https://en.wikipedia.org/wiki/Methylamine">Metilamina</a>
        </td>
        <td style="text-align: center;">
            <a href="https://en.wikipedia.org/wiki/Sulfuric_acid">Ácido Sulfúrico</a>
        </td>
     </tr>
         <tr>
        <td>
          <img src="dados/escatol.png"></img>
        </td>
        <td>
          <img src="dados/fenilacetona.png"></img>
        </td>
        <td>
          <img src="dados/oleico.png"></img>
        </td>
     </tr>
     <tr>
        <td style="text-align: center;">
            <a href="https://en.wikipedia.org/wiki/Skatole">Escatol</a>
        </td>
        <td style="text-align: center;">
            <a href="https://en.wikipedia.org/wiki/Phenylacetone">Fenilacetona</a>
        </td>
        <td style="text-align: center;">
            <a href="https://en.wikipedia.org/wiki/Oleic_acid">Ácido Oleico</a>
        </td>
     </tr>
</table>

# 2. Breaking DES

Vamos tentar descriptografar os seguintes dados:
```python
cifra = b"\x8a\xa8!hpzjU&\xc6\x07#Z\x8d\xa9\xdd\xbf_=\x0fe\x8eV\xfa\xb6W\xf61;8\r\x8el\xd2\xab\x8eD\xc2\xc0\xac\x8cM\xe6\x8eK\x04#z[\x84q\x81\xfbc\xd3\xd7"
```

Podemos criar uma chave aleatória e ver no que dá:
```python
chave = b"mudar123"
```

In [ ]:
cifra = b'\x8a\xa8!hpzjU&\xc6\x07#Z\x8d\xa9\xdd\xbf_=\x0fe\x8eV\xfa\xb6W\xf61;8\r\x8el\xd2\xab\x8eD\xc2\xc0\xac\x8cM\xe6\x8eK\x04#z[\x84q\x81\xfbc\xd3\xd7'
chave = b"mudar123"

des = DES.new(chave, DES.MODE_ECB)
des.decrypt(cifra)

Bom, o resultado não está nem perto de alguma mensagem inteligivel. Muito provavelmente não é essa a chave.

Para quebrar o DES, vamos nos aproveitar do fato de que o espaço das chaves é relativamente pequeno (72 quadrilhões) e tentar cada chave possível.

...mentira, vamos reduzir ainda mais o espaço das chaves para poder rodar em tempo hábil. Vamos considerar que a chave contém somente letras maiusculas e números. Também vamos considerar que a chave tem no máximo 5 caracteres. Isso vai reduzir o espaço das chaves pra perto de 60 milhões (36^5).

### Exercício 2

A função `gerar_chave` retorna todas as chaves possíveis de um dado `dicionario`. Verifique alguns dados da função `gerar_chave`. Espero que você se convença de que ela realmente gera todas as chaves.

Teste algumas chaves geradas pela função e tente descriptografar a `cifra`.

A função `pad` serve apenas para garantir que as mensagens tenham um comprimento múliplo de 8 bytes (64 bits), que é o tamanho do bloco para o DES.

In [ ]:
from itertools import product

def pad(data, mod=8):
    """
    Garante que `data` sempre tem comprimento multiplo de `mod`.
    """
    padding = b" "*((mod - (len(data) % mod)) % mod)
    return data + padding

def gerar_chave(alfabeto, comprimento_maximo):
    """
    Gera todas as chaves possíveis de um dado `alfabeto`
    dado um `comprimento_máximo`. 
    """
    for i in range(1, comprimento_maximo+1):
        for c in product(alfabeto, repeat=i):
            yield "".join(c).encode("ascii")
            
alfabeto = "HABCDEFGIJKLMONPQSRTUVWXYZ0123456789"
chaves = gerar_chave(alfabeto, 6)

# IMPLEMENTE A PARTIR DAQUI!


Vamos, então, tentar todas as chaves pra tentar encontrar o texto.

### Exercício 3

Quebre a mensagem DES:
```python
cifra = b'\x8a\xa8!hpzjU&\xc6\x07#Z\x8d\xa9\xdd\xbf_=\x0fe\x8eV\xfa\xb6W\xf61;8\r\x8el\xd2\xab\x8eD\xc2\xc0\xac\x8cM\xe6\x8eK\x04#z[\x84q\x81\xfbc\xd3\xd7'
```
tentando todas as chaves possíveis.

*dica 1: A mensagem contém apenas caracteres legíveis.*

*dica 2: Não é preciso tentar todas as combinações. É possível terminar o loop assim que uma mensagem for encontrada.*

In [ ]:
from tqdm.notebook import tqdm

def breaking_des(cifra, alfabeto):
    """
    Quebra a `cifra` DES com força bruta dado um `alfabeto`.
    A função não retorna nada, apenas printa a chave e o 
    texto decifrado
    """
    total_chaves = sum([len(alfabeto) ** i for i in range(5+1)]) - 1
    # Este loop pass por todas as chaves em `alfabeto`.
    for chave in tqdm(gerar_chave(alfabeto, 5), 
                      total=total_chaves,
                      desc="Breaking DES"):
        # IMPLEMENTE A PARTIR DAQUI!
        

    # FIM DA IMPLEMENTAÇÃO

alfabeto = "HABCDEFGIJKLMONPQSRTUVWXYZ0123456789"
cifra = b'\x8a\xa8!hpzjU&\xc6\x07#Z\x8d\xa9\xdd\xbf_=\x0fe\x8eV\xfa\xb6W\xf61;8\r\x8el\xd2\xab\x8eD\xc2\xc0\xac\x8cM\xe6\x8eK\x04#z[\x84q\x81\xfbc\xd3\xd7'
breaking_des(cifra, alfabeto)

.

.

.

.

.

.

.

.

### UOOOU!

![yeah science](dados/yeah_science.gif)

É claro que estamos reduzindo muito o espaço de busca, mas acredite que é uma estratégia viável.
Em 1999, foi criada uma máquina ([EFF DES Cracker](https://en.wikipedia.org/wiki/EFF_DES_cracker)) capaz de tentar todas as chaves possíveis em aproximadamente 9 dias.

![DES Cracker](dados/Paul_kocher_deepcrack.jpg)

Hoje em dia é possível comprar hardware capaz de testar todas as combinações em 26 horas: https://crack.sh/

O DES, então, é considerado inseguro para criptografarmos dados.

Será que connseguimos descriptografar os dados enviados pelo Gus dessa maneira?

### Exercício 4

Tente quebrar as mensagens recebidas do Gus usando força bruta. (Obs: não vai dar).

In [ ]:
cifra_gus1 = b'\x1c\xce\x12\xbagH7j\x03\xb1\xe7\x9a+p\xe0\x0b\xb8\r\xf5\\\xeb\xd2l\xab\xda\xa704\x9a\xa3\xba\nm4"$n\xfb\xfd\xaa\xe8\x88\x15ek.\xc8y\x05\x1c\x86w\xe1\xca"\xf0\xba\xe3\xe0\x1eF\x88/\xc9\xba\xcfB\x15\xfc\x9a\x0f)'
texto_gus1 = pad(b"Uma galinha fatiada, creme de leite de ganso manso, sal, pimenta, ...")

# IMPLEMENTE A PARTIR DAQUI! cifra_gus1


In [ ]:
cifra_gus2 = b'\xd6\xfdwc\xf4\xa2I\x16\x82\xb3`\x06\xcbO\x07i\x8d\x89\x88c\x17\xacV?;\xf5|\x0f\x1ab\x95\x14\x01k\x1fW(<\xca\x19T\x8b\x11\xe1\xa0x\xae(|\xf6\xbc\xf1\xc1\xb4\x8d\x11\xe2\xe4\x80\x8d\xd97\x1c\xa5UW\x97\xf7r\xcb\x98\xc51K\x1bv\xd6\x8av\xd8\x15\xe3X"\xa6\x9a*&'

# IMPLEMENTE A PARTIR DAQUI! cifra_gus2


Nada... 

Não é possível quebrar essas mensagens dessa forma.

Talvez o Gus seja um pouco mais esperto do que esperávamos...

# 3. Double DES
Bom, se o DES não resiste a ataques de força bruta, o que aconteceria se criptografássemos com o DES duas vezes com chaves *chave_a* e *chave_b*? Certamente teremos um sistema mais seguro, certo? Criptografar um texto duas vezes com DES é conhecido como **Double DES** ou **DDES**.

![DDES](dados/DDES-block.png)

### Exercício 5
Implemente duas funções

a) Uma para criptografar com o DDES, recebendo três parâmetros: `texto`, `chave_a` e `chave_b` e que retorna `cifra`.

b) Uma para descriptografar com o DDES, recebendo três parâmetros: `cifra`, `chave_a` e `chave_b` e que retorna `texto`.

In [ ]:
def double_des_enc(texto, chave_a, chave_b):
    """
    Criptografa `texto` com DDES dados `chave_a` e
    `chave_b`
    """
    # IMPLEMENTE A PARTIR DAQUI!

    # FIM DA IMPLEMENTAÇÃO

def double_des_dec(cifra, chave_a, chave_b):
    """
    Desriptografa `texto` com DDES dados `chave_a` e
    `chave_b`
    """
    # IMPLEMENTE A PARTIR DAQUI!

    # FIM DA IMPLEMENTAÇÃO

chave_a = b"H2SO4"  # Ácido Sulfurico
chave_b = b"NaOH" # Hidróxido de sódio
texto = pad(b"C21H26O2")  # 🌿
print("texto:", texto)

cifrado = double_des_enc(texto, chave_a, chave_b)
print("cifra DDES:", cifrado)
assert cifrado ==  b'\xec\xfe\xa4mY\xaa\xbf\x7f', "Erro no exercício a"

decifrado = double_des_dec(cifrado, chave_a, chave_b)
print("decifrado:", decifrado)
assert decifrado ==  texto, "Erro no exercício b"

Para quebrar uma mensagem *cifra2* do DDES, precisaríamos:
1. Gerar **2^56** *cifra1* de todas as *chave_b* possíveis.
2. Para cada uma das cifras geradas anteriormente, gerar o *texto* usando todas as **2^56** *chave_a* possíveis.

Utilizando o melhor hardware que temos hoje, que leva 26 horas para tentar todas as chaves, precisariamos de *10^14* anos para quebrar o DDES, aproximadamente 10 mil vezes a idade do universo.

A fragilidade do DES está no fato de que a chave é muito pequena (56 bits) e pode ser quebrada na força bruta. Resolvemos esse problema aplicando a criptografia duas vezes! O que é equivalente a aumentar a chave para 112 bits.

Se o Gus criptografou a mensagem com DDES, não vamos consegir decifrar, certo?

![ate!](dados/heman.jpg)
.

.

.

.

.

.

.

.

.

![errado!](dados/achou-errado-otario.jpg)


# 4. Meet 'Meet-in-the-middle'

Quebrar o DDES realmente levaria *10^14* anos, se só tivessemos cifras em mãos. Mas o que poderiamos fazer se obtivermos um par de *cifra* e *texto*?

Nós temos uma mensagem do Gus que já sabemos, que é uma receita de frango frito. Seria possível recuperar alguma informação de lá para ajudar a descriptografar a outra mensagem?

Alguma sugestão?

![tempo](dados/tempo.gif)

.

.

.

.

.

Com apenas um par de dados (cifra/texto), é possível atacar o sistema com o *meet-in the-middle-attack*. Nós conseguimos reduzir o problema de uma chave de 112 bits (*10^14* anos) para um problema de 57 bits (2 dias).

Ataques baseados em pelo menos um par *cifra/texto* são chamados de *known plaintext attack*.


### Como funciona?

Para atacar o DDES, vamos partir da seguinte premissa:

> Criptografando o *texto* com a *chave a* resulta em um estado intermediário *middle*, que é equivalente a descriptografar a *cifra* com a *chave b*.

![MITM](dados/MITM.png)

Sabendo disso, podemos: 
1. Criptografar o *texto* conhecido com todas as *2^56* chaves, armazenando os estados *middle* e chaves em *middle_a*.
2. Descriptografar a *cifra* conhecida com todas as 2^56 chaves, armazenando os estados *middle* e chaves em *middle_b*.
3. Procurar os estados *middle* de *middle_a* em *middle_b*. Se encontrarmos estados iguais, as chaves desses estados correspondem às chaves utilizadas para criptografar os dados.

Esse ataque é conhecido como *Meet-in-the-middle-attack*.

> Quem propôs o *meet in middle* foram Diffie e Hellman, os inventores da [criptografia assimétrica](https://en.wikipedia.org/wiki/Public-key_cryptography), que está rodando no seu navegador neste  instante.

#### Bora implementar!

Bom, como estamos com limitação de CPU e RAM aqui, vamos reduzir ainda mais o espaço das chaves. Vamos considerar que a chave tem apenas os digitos de `0-9` e as letras `HCNOPS`, e um comprimento de até 6 caracteres.

Vamos tentar encontrar as chaves a partir do par de mensagens que o Hank interceptou:

### Exercício 6

O recheio da função `breaking_ddes` precisa de alguns detalhes para que consigamos realizar o ataque.

Para cada chave do loop:

a) Crie uma instância do objeto DES da `chave`.

b) Criptografe o `texto` na variável `ma` e descriptografe a `cifra` na variável `mb`.

c) Armazene a chave criptográfica no dicionário `middle_a` e `middle_b`. A indexação deve ser o estado intermediário `middle` (`ma` e `mb`).

d) Busque no dicionário `middle_a` pelo estado intermediário `mb`. Se o estado for encontrado, retorne as chaves `middle_a[mb]` e `middle_b[mb]`.

d) Busque no dicionário `middle_b` pelo estado intermediário `ma`. Se o estado for encontrado, retorne as chaves `middle_a[ma]` e `middle_b[ma]`.

In [ ]:
def breaking_ddes(texto, cifra, alfabeto):
    total_chaves = sum([len(alfabeto) ** i for i in range(6+1)]) - 1
    middle_a = {}
    middle_b = {}
    for chave_cripto in tqdm(gerar_chave(alfabeto, 6), 
                      total=total_chaves, 
                      desc="Breaking DDES"):  
        # IMPLEMENTE A PARTIR DAQUI!
        # Exercício 6.a.
        
        # Exercício 6.b.
        
        # Exercício 6.c.
        
        # Exercício 6.d.

        # Exercício 6.e.

        # FIM DA IMPLEMENTAÇÃO

    return None, None

cifra_gus1 = b'\x1c\xce\x12\xbagH7j\x03\xb1\xe7\x9a+p\xe0\x0b\xb8\r\xf5\\\xeb\xd2l\xab\xda\xa704\x9a\xa3\xba\nm4"$n\xfb\xfd\xaa\xe8\x88\x15ek.\xc8y\x05\x1c\x86w\xe1\xca"\xf0\xba\xe3\xe0\x1eF\x88/\xc9\xba\xcfB\x15\xfc\x9a\x0f)'
texto_gus1 = pad(b"Uma galinha fatiada, creme de leite de ganso manso, sal, pimenta, ...")

cifra_gus2 = b'\xd6\xfdwc\xf4\xa2I\x16\x82\xb3`\x06\xcbO\x07i\x8d\x89\x88c\x17\xacV?;\xf5|\x0f\x1ab\x95\x14\x01k\x1fW(<\xca\x19T\x8b\x11\xe1\xa0x\xae(|\xf6\xbc\xf1\xc1\xb4\x8d\x11\xe2\xe4\x80\x8d\xd97\x1c\xa5UW\x97\xf7r\xcb\x98\xc51K\x1bv\xd6\x8av\xd8\x15\xe3X"\xa6\x9a*&'

alfabeto = "HCONPS9876453210"
chave_a, chave_b = breaking_ddes(texto_gus1, cifra_gus1, alfabeto)
if chave_a is not None:
    print("Achou!")
    print("chave_a:", chave_a)
    print("chave_b:", chave_b)

### Exercício 7

Com as chaves encontradas, decifre a segunda mensagem do Gus.

In [ ]:
# IMPLEMENTE A PARTIR DAQUI!


.

.

.

.

.
### Já era Gus! BUSTED!

![gus death](dados/gus-death.jpg)

Quebramos o DDES usando *meet-in-the-middle*, um tipo de *known 
plaintext attack*. O DDES, portanto, também não é seguro.

*Obs: Uma das chaves na verdade é C9H10O (fenilacetona). A chave C9H11O obtém o mesmo texto, pois o DES usa apenas 56 dos 64 bits das chaves. Veja [key schedule](https://en.wikipedia.org/wiki/Data_Encryption_Standard#Key_schedule).*

> Pra lembrar:
> * Não use DES nem DDES. Existe o Triple DES (3DES), que é considerado seguro, mas é superlento.
> * Aliás, não criptografe duas vezes achando que vai tornar um sistema mais seguro.
> * Não use sistemas criptográficos com chaves pequenas.
> * Joguem CTF!
> * *Não fabrique, não venda e não use drogas.*

![proerd](dados/proerd.jpg)


*Esse notebook foi inspirado em um desafio do https://picoctf.org/*